## State of North Dakota Approachable AI - Cognitive Services

This demonstration acts as an introduction to Microsoft Cognitive Services. Unlike the custom models we have built thus far, these are highly advanced, specialized models that have been produced by Microsoft research engineers.

These models solve specialized problems - but are state of the art at solving those problems. As a general piece of advice, if a cognitive services model can be used as part of your data science workflow, then it is ususually worth doing so.

In [ ]:
import requests # Important library that we use whenever making an API call to a website
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image # Python imaging library
from io import BytesIO
import os
import sys

%matplotlib inline

In [ ]:
# Extract our secret key and the API endpoint from the system
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']

ocr_url = endpoint + "vision/v2.0/ocr"

In [ ]:
# Set image_url to the URL of an image that you want to analyze.
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/" + \
    "Atomist_quote_from_Democritus.png/338px-Atomist_quote_from_Democritus.png"

### Example 1: A simple quote from wikimedia commons

In [ ]:
# Set the subscription key - we do this in a JSON header
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

# Set the optional parameters that we want the model to interpret - unk is unknown
params = {'language': 'unk', 'detectOrientation': 'true'}

# Define the data - in this case it is coming from a URL, not from storage
data = {'url': image_url}

In [ ]:
# Send the JSON message to the service
response = requests.post(ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()

# Get the result of the model
analysis = response.json()
print(analysis)

In [ ]:
# Extract the word bounding boxes and text.

def extract_info(analysis):
    line_infos = [region["lines"] for region in analysis["regions"]]
    word_infos = []
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                word_infos.append(word_info)
    return word_infos

word_infos = extract_info(analysis)
word_infos

In [ ]:
# Display the image and overlay it with the extracted text.
plt.figure(figsize=(5, 5))
image = Image.open(BytesIO(requests.get(image_url).content))
ax = plt.imshow(image, alpha=0.5)
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch = Rectangle(origin, bbox[2], bbox[3],
                      fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=20, weight="bold", va="top")
plt.axis("off")
plt.show()

### Example 2: Use a local image

In [ ]:
# Get the image path
image_path = "../data/demo-label.jpg"

# Read the image into a byte array
image_data = open(image_path, "rb").read()

# Set Content-Type to octet-stream - format needed for the model
headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}

# Set the optional parameters that we want the model to interpret - unk is unknown
params = {'language': 'unk', 'detectOrientation': 'true'}

# put the byte array into your post request
response = requests.post(ocr_url, headers=headers, params=params, data = image_data)
response.raise_for_status()

# Get the result of the model
analysis = response.json()
analysis

In [ ]:
# Extract the results from the response
word_infos = extract_info(analysis)
word_infos

In [ ]:
# Display the image and overlay it with the extracted text.
plt.figure(figsize=(15, 15))
image = Image.open(image_path)
ax = plt.imshow(image, alpha=0.5)
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch = Rectangle(origin, bbox[2], bbox[3],
                      fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=20, weight="bold", va="top")
plt.axis("off")
plt.show()

In [ ]:
# Apply a translation and rotation to the image
from scipy import ndimage
image_angle = - analysis["textAngle"]
image_offset = (50, 130)

plt.figure(figsize=(15, 15))
image = Image.open(image_path)
rotated_image = ndimage.rotate(image, image_angle * 360 / 6.28)

ax = plt.imshow(rotated_image, alpha=0.5)
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0] + image_offset[0], bbox[1] + image_offset[1])
    patch = Rectangle(origin, bbox[2], bbox[3],
                      fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=20, weight="bold", va="top")
plt.axis("off")
plt.show()